In [1]:
# Main import block
import asyncio
import datetime
import signal
import time
import warnings
from contextlib import contextmanager
from datetime import date, datetime
from io import StringIO

# Sneaky scrapper
import cfscrape
# Upload to google sheets
import gspread
# import df2gspread as d2g
import gspread_pandas as gsp
import numpy as np
import pandas as pd
import requests
# Upload to yandex
import yadisk
from bs4 import BeautifulSoup
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import httpx

warnings.filterwarnings("ignore")

In [2]:
url = "https://www.lme.com/Metals/Non-ferrous#tabIndex=1"
service = Service()
driver = webdriver.Chrome(service=service)

driver.maximize_window()
driver.get(url=url)

time.sleep(2)

driver.execute_script("window.scrollTo(0, window.scrollY + window.innerHeight);")
driver.execute_script("window.scrollTo(0, window.scrollY + window.innerHeight);")

time.sleep(2)


html_code = driver.page_source
driver.close()

In [3]:
soup = BeautifulSoup(html_code, "html.parser")
# Вытаскиваем данные из табличного блока
data_raw = soup.find_all("div", class_="metal-block-row__blocks")

In [4]:
data_raw

[<div class="metal-block-row__blocks"><a class="metal-block metal-block--init" data-block-id="1b72897c-6a30-480c-a74a-cfc2faaaff16" data-day-delayed="True" href="/en/Metals/Non-ferrous/LME-Aluminium"><div class="metal-block__inner"><h4 class="metal-block__title"><span class="metal-block__title-text">
                     LME Aluminium
                 </span></h4> <span class="metal-block__price-wrapper"><span class="metal-block__price">2703.00</span></span> <span class="metal-block__change-wrapper"><span class="metal-block__change metal-block__change--up">
                     + 0.11%
                 </span></span> <span class="metal-block__type">Official Price</span> <!-- --></div></a> <a class="metal-block metal-block--init" data-block-id="fc09fcde-6438-4834-9221-98da9ed54eea" data-day-delayed="True" href="/en/Metals/Non-ferrous/LME-Copper"><div class="metal-block__inner"><h4 class="metal-block__title"><span class="metal-block__title-text">
                     LME Copper
         

In [5]:
# Приводим к тексту
metalls_raw = data_raw[0].text

# Убираем все лишнее
metalls_raw = metalls_raw.replace(" ", "").replace("LME", "LME_").split("\n")
metalls_raw = " ".join(metalls_raw).strip().split(" ")
# Конфигурируем новый лист по металлам
metalls_raw = [metalls_raw[i : i + 4] for i in range(0, len(metalls_raw), 4)]
# Обрезаем до нужной инфы
metalls_raw = [i[:2] for i in metalls_raw]
# Приведем цены к числовому формату
for metall in metalls_raw:
    metall[1] = pd.to_numeric(metall[1])

# Формируем строку датафрейма по металлам
dict_list = [{item[0]: item[1]} for item in metalls_raw]

metall_df = pd.DataFrame()

for i in dict_list:
    keys = list(i.keys())
    values = list(i.values())
    metall_df[keys[0]] = values

# Дропаем ненужные колонки
metall_df.drop(columns=["LME_AluminiumAlloy", "LME_NASAAC"], inplace=True)
metall_df = metall_df.rename(
    columns={
"LME_Aluminium": "aluminium",
"LME_Copper": "copper",
"LME_Lead": "lead",
"LME_Nickel": "nickel",
"LME_Zinc": "zink",
"LME_Tin": "tin",
    }
)

# Достанем в вставим дату
date_raw = soup.find_all("span", class_="metal-block-container__refreshed-on")
date_raw = date_raw[0]
date_raw = str(date_raw).split(">")[1]
date_raw = date_raw.replace("\xa0", "").replace("</span", "")
date_raw = pd.to_datetime(date_raw) - pd.Timedelta(days=1)

metall_df["date"] = date_raw

# Поменяем порядок колонок
new_column_order = [
    "date",
    "aluminium",
    "copper",
    "lead",
    "nickel",
    "zink",
    "tin",
]
metall_df = metall_df[new_column_order]


# Сохраняем полученные результаты
lme_db = pd.read_excel("./data/LME_db_new.xlsx", index_col=0)
lme_db = pd.concat([lme_db, metall_df]).reset_index(drop=True)

# Дропаем дубликаты
lme_db = lme_db.drop_duplicates(subset=lme_db.columns.to_list()[1:])
lme_db

,date,aluminium,copper,lead,nickel,zink,tin
0,2022-01-04,2815.5,9660.0,2327.0,20730.0,3602.0,39400.0
1,2022-01-05,2866.0,9778.0,2343.0,20900.0,3660.0,39700.0
2,2022-01-06,2912.5,9565.0,2291.0,20480.0,3590.0,39650.0
3,2022-01-07,2919.5,9615.0,2342.0,20725.0,3602.5,40650.0
4,2022-01-10,2923.0,9665.0,2303.0,21045.0,3576.5,40425.0
...,...,...,...,...,...,...,...
943,2025-09-29,2672.0,10233.0,1957.0,15065.0,2974.5,34755.0
944,2025-09-30,2668.5,10300.0,1957.0,15075.0,3010.0,35285.0
945,2025-10-01,2683.5,10263.0,1958.0,14965.0,3027.0,36000.0
946,2025-10-02,2700.0,10455.0,1987.5,15110.0,3102.0,36375.0


In [14]:
soup.find_all("span", class_="metal-block-container__refreshed-on")

[<span class="metal-block-container__refreshed-on"> 16 Nov 2024</span>]

In [40]:
async def lme_selenium_async():
    url = "https://www.lme.com/Metals/Non-ferrous#tabIndex=1"

    service = Service()
    driver = webdriver.Chrome(service=service)

    try:
        driver.maximize_window()
        driver.get(url=url)

        time.sleep(2)

        driver.execute_script(
            "window.scrollTo(0, window.scrollY + window.innerHeight);"
        )
        driver.execute_script(
            "window.scrollTo(0, window.scrollY + window.innerHeight);"
        )

        time.sleep(2)

        html_code = driver.page_source

        soup = BeautifulSoup(html_code, "html.parser")
        # Вытаскиваем данные из табличного блока
        data_raw = soup.find_all("div", class_="metal-block-row__blocks")

        # Приводим к тексту
        metalls_raw = data_raw[0].text

        # Убираем все лишнее
        metalls_raw = metalls_raw.replace(" ", "").replace("LME", "LME_").split("\n")
        metalls_raw = " ".join(metalls_raw).strip().split(" ")

        # Конфигурируем новый лист по металлам
        metalls_raw = [metalls_raw[i : i + 4] for i in range(0, len(metalls_raw), 4)]

        # Обрезаем до нужной инфы
        metalls_raw = [i[:2] for i in metalls_raw]

        # Приведем цены к числовому формату
        for metall in metalls_raw:
            metall[1] = pd.to_numeric(metall[1])

        # Формируем строку датафрейма по металлам
        dict_list = [{item[0]: item[1]} for item in metalls_raw]

        metall_df = pd.DataFrame()

        for i in dict_list:
            keys = list(i.keys())
            values = list(i.values())
            metall_df[keys[0]] = values

        # Дропаем ненужные колонки
        metall_df.drop(columns=["LME_AluminiumAlloy", "LME_NASAAC"], inplace=True)
        metall_df = metall_df.rename(
            columns={
                "LME_Aluminium": "aluminium",
                "LME_Copper": "copper",
                "LME_Lead": "lead",
                "LME_Nickel": "nickel",
                "LME_Zinc": "zink",
                "LME_Tin": "tin",
            }
        )

        # Достанем в вставим дату
        date_raw = soup.find_all("span", class_="metal-block-container__refreshed-on")
        date_raw = date_raw[0]
        date_raw = str(date_raw).split(">")[1]
        date_raw = date_raw.replace("\xa0", "").replace("</span", "")
        date_raw = pd.to_datetime(date_raw) - pd.Timedelta(days=1)

        metall_df["date"] = date_raw

        # Поменяем порядок колонок
        new_column_order = [
            "date",
            "aluminium",
            "copper",
            "lead",
            "nickel",
            "zink",
            "tin",
        ]
        metall_df = metall_df[new_column_order]

        # Сохраняем полученные результаты
        lme_db = pd.read_excel("./data/LME_db_new.xlsx", index_col=0)
        lme_db = pd.concat([lme_db, metall_df]).reset_index(drop=True)

        # Дропаем дубликаты
        lme_db = lme_db.drop_duplicates(subset=lme_db.columns.to_list()[1:])

        # Сохраняем
        with pd.ExcelWriter(
            "./data/LME_db_new.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            lme_db.to_excel(writer, sheet_name="LME_main")
        
        print("LME_main is done!!!")
        display(lme_db.tail(10))

    except Exception as error:
        print(error)

    finally:
        driver.close()
        driver.quit()


In [41]:
################################################################
#########################   KITCO (main) ########################
################################################################


# Ассинхронный парсер Kitco
async def kitco_parser_async():
    async def page_getter():
        try:
            url = "https://www.kitco.com/price/fixes/london-fix"

            service = Service()
            driver = webdriver.Chrome(service=service)
            # driver.maximize_window()
            driver.get(url=url)
            #time.sleep(2)

            driver.execute_script(
                "window.scrollTo(0, window.scrollY + window.innerHeight);"
            )
            driver.execute_script(
                "window.scrollTo(0, window.scrollY + window.innerHeight);"
            )

            time.sleep(2)

            html_code = driver.page_source
            
            return html_code

        except Exception as error:
            print(error)

        finally:
            driver.close()
            driver.quit()
            
    
    html_code = await page_getter()
    
    try:
        # Работа над полученными данными
        soup = BeautifulSoup(html_code)

        # Выделим данные и предобработаем для получения дневных данных
        day = soup.find_all("div", class_="border")[1]
        for i in day:
            day = list(i.find_all("div"))

        day = day[4:]

        day = [str(i) for i in day]

        day = [i.replace("<div>", "").replace("</div>", "") for i in day]

        day[0] = pd.to_datetime(day[0])

        day[1] = day[1].split("/")
        day[1] = day[1][1].strip().replace(",", "")
        day[1] = pd.to_numeric(day[1])

        day[2] = pd.to_numeric(day[2])

        day[3] = day[3].split("/")
        day[3] = day[3][1].strip().replace(",", "")
        day[3] = pd.to_numeric(day[3])

        day[4] = day[4].split("/")
        day[4] = day[4][1].strip().replace(",", "")
        day[4] = pd.to_numeric(day[4])

        df_row = (
            pd.DataFrame([day], columns=["Date", "Gold", "Silver", "Platinum", "Palladium"])
            .sort_values("Date")
            .reset_index(drop=True)
        )

        # Выделим данные и предобработаем для получения исторических данных
        historical = soup.find_all("div", class_="border")[2]
        for i in historical:
            historical = list(i.find_all("div"))

        historical = historical[5:]

        historical = [str(i) for i in historical]

        historical = [i.replace("<div>", "").replace("</div>", "") for i in historical]
        historical = [item for item in historical if "<div" not in item]

        # Разобьем по дням
        historical = [historical[i : i + 5] for i in range(0, len(historical), 5)]

        # Обработаем данные внутри каждого дня
        for i in historical:
            i[0] = pd.to_datetime(i[0])

            i[1] = i[1].split("/")
            i[1] = i[1][1].strip().replace(",", "")
            i[1] = pd.to_numeric(i[1])

            i[2] = pd.to_numeric(i[2])

            i[3] = i[3].split("/")
            i[3] = i[3][1].strip().replace(",", "")
            i[3] = pd.to_numeric(i[3])

            i[4] = i[4].split("/")
            i[4] = i[4][1].strip().replace(",", "")
            i[4] = pd.to_numeric(i[4])

        df_historical = (
            pd.DataFrame(
                historical, columns=["Date", "Gold", "Silver", "Platinum", "Palladium"]
            )
            .sort_values("Date")
            .reset_index(drop=True)
        )

        df_historical = pd.concat([df_historical, df_row]).reset_index(drop=True)
        kitko_db = pd.read_excel("./data/kitko_db.xlsx", index_col=0)

        kitko_db = (
            pd.concat([kitko_db, df_historical])
            .reset_index(drop=True)
            .drop_duplicates(subset=["Date"])
        )

        with pd.ExcelWriter(
            "./data/kitko_db.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            kitko_db.to_excel(writer, sheet_name="kitco_metall")

        print("KITKO_main is done!!!")
        display(kitko_db.tail(10))
        
    except Exception as e:
        print(e)

In [2]:
url = "https://www.kitco.com/price/fixes/london-fix"
service = Service()
driver = webdriver.Chrome(service=service)
# driver.maximize_window()
driver.get(url=url)
#time.sleep(2)

driver.execute_script(
    "window.scrollTo(0, window.scrollY + window.innerHeight);"
)
driver.execute_script(
    "window.scrollTo(0, window.scrollY + window.innerHeight);"
)

time.sleep(2)

html_code = driver.page_source

In [3]:
soup = BeautifulSoup(html_code)

In [4]:
day = soup.find_all("div", class_="border")[1]
for i in day:
    day = list(i.find_all("div"))

In [5]:
day

[<div class="min-w-[700px]"><div class="grid grid-cols-5 border-b border-gray-200 text-center text-base font-semibold"><div class="flex items-center justify-center">DATE</div><h4>Gold <br/> <span class="text-sm text-gray-600 font-normal">AM/PM</span></h4><h4>Silver <br/> <span class="text-sm text-gray-600 font-normal">NOON</span></h4><h4>Platinum <br/> <span class="text-sm text-gray-600 font-normal">AM/PM</span></h4><h4>Palladium <br/> <span class="text-sm text-gray-600 font-normal">AM/PM</span></h4></div><div class="grid grid-cols-5 border-b border-gray-200 text-center border-[rgba(0,0,0,0)] py-4 undefined bg-alt-row"><div>January 03, 2025</div><div>2,654.30 / 2,646.80</div><div>29.82</div><div>934.00 / 941.00</div><div>929.00 / 931.00</div></div></div>,
 <div class="grid grid-cols-5 border-b border-gray-200 text-center text-base font-semibold"><div class="flex items-center justify-center">DATE</div><h4>Gold <br/> <span class="text-sm text-gray-600 font-normal">AM/PM</span></h4><h4>Si

In [6]:
# Правки 7 ноября
soup = BeautifulSoup(html_code)

day = soup.find_all("div", class_="grid")

# Вычленение первичного контента
day_content = []
for i in range(len(day)):
    day_content.append(day[i].contents)

# Первичное преобразование
for container in day_content:
    for element in range(len(container)):
        try:
            container[element] = (
                str(container[element])
                .replace("<div>", "")
                .replace("</div>", "")
                .replace("<!-- -->", "")
            )
            container[element]
        except:
            pass

# Удалим неинформативные строки
day_content = day_content[1:]
day_content.pop(1)

# Преобразование дат и вычлинение значений
for element in day_content:
    # Преобразованиие даты
    element[0] = pd.to_datetime(element[0]).strftime("%Y-%m-%d")

    # Числовые преобразования
    element[1] = pd.to_numeric(element[1].split("/")[1].strip().replace(",", ""))
    element[2] = pd.to_numeric(element[2].split("/")[0].strip().replace(",", ""))
    element[3] = pd.to_numeric(element[3].split("/")[1].strip().replace(",", ""))
    element[4] = pd.to_numeric(element[4].split("/")[1].strip().replace(",", ""))

# Формируем датафрейм
day_content = pd.DataFrame(
    day_content, columns=["Date", "Gold", "Silver", "Platinum", "Palladium"]
)

# Преобразуем типы
day_content["Date"] = pd.to_datetime(day_content["Date"])
day_content = day_content.sort_values("Date").reset_index(drop=True)

# Возьмем последние 5 строк
day_update = day_content.tail()

# Склеим с историческими данными
historical_df = pd.read_excel("./data/kitko_db.xlsx", index_col=0)
historical_df = (
    pd.concat([historical_df, day_update], axis=0)
    .reset_index(drop=True)
    .drop_duplicates()
)

with pd.ExcelWriter(
    "./data/kitko_db.xlsx",
    date_format="YYYY-MM-DD",
    datetime_format="YYYY-MM-DD",
) as writer:
    historical_df.to_excel(writer, sheet_name="kitco_metall")

2660.2

In [75]:
# Выделим данные и предобработаем для получения дневных данных
day = soup.find_all("div", class_="border")[1]
for i in day:
    day = list(i.find_all("div"))

day = day[4:]

day = [str(i) for i in day]

day = [i.replace("<div>", "").replace("</div>", "") for i in day]

for i in day:
    if type(i) == str:
        i = i.replace("<!-- -->", "")
    else:
        pass

day[0] = pd.to_datetime(day[0])

day[1] = day[1].split("/")
day[1] = day[1][1].strip().replace(",", "")
day[1] = pd.to_numeric(day[1])

day[2] = pd.to_numeric(day[2])

day[3] = day[3].split("/")
day[3] = day[3][1].strip().replace(",", "")
day[3] = pd.to_numeric(day[3])

day[4] = day[4].split("/")
day[4] = day[4][1].strip().replace(",", "")
day[4] = pd.to_numeric(day[4])

df_row = (
    pd.DataFrame([day], columns=["Date", "Gold", "Silver", "Platinum", "Palladium"])
    .sort_values("Date")
    .reset_index(drop=True)
)

# Выделим данные и предобработаем для получения исторических данных
historical = soup.find_all("div", class_="border")[2]
for i in historical:
    historical = list(i.find_all("div"))

historical = historical[5:]

historical = [str(i) for i in historical]

historical = [i.replace("<div>", "").replace("</div>", "") for i in historical]
historical = [item for item in historical if "<div" not in item]

# Разобьем по дням
historical = [historical[i : i + 5] for i in range(0, len(historical), 5)]

DateParseError: Unknown datetime string format, unable to parse: <div class="grid grid-cols-5 border-b border-gray-200 text-center border-[rgba(0,0,0,0)] py-4 undefined bg-alt-row">November 05, 20242,738.75 / <!-- -->2,742.5532.65995.00 / <!-- -->1,001.001,087.00 / <!-- -->1,092.00, at position 0

In [10]:
day

[Timestamp('2024-10-07 00:00:00'),
 '<!-- -->2640.95',
 '31.88',
 '986.00 / <!-- -->980.00',
 '1,021.00 / <!-- -->1,011.00']

In [17]:
day[1].replace("<!-- -->", "")

'2640.95'

In [21]:
day

[Timestamp('2024-10-07 00:00:00'),
 '<!-- -->2640.95',
 '31.88',
 '986.00 / <!-- -->980.00',
 '1,021.00 / <!-- -->1,011.00']

In [34]:
for i in day:
    if type(i) == str:
        i = i.replace("<!-- -->", "")
    else:
        pass


In [35]:
day

[Timestamp('2024-10-07 00:00:00'),
 '<!-- -->2640.95',
 '31.88',
 '986.00 / <!-- -->980.00',
 '1,021.00 / <!-- -->1,011.00']

In [37]:
day[1] = day[1].replace("<!-- -->", "")

In [38]:
day

[Timestamp('2024-10-07 00:00:00'),
 '2640.95',
 '31.88',
 '986.00 / <!-- -->980.00',
 '1,021.00 / <!-- -->1,011.00']

In [13]:
################################################################
#########################   KITCO (new) ########################
################################################################

async def kitco_parser_async():
    url = "https://www.kitco.com/price/fixes/london-fix"

    async def get_raw_data(url):
        async with httpx.AsyncClient() as client:
            response = await client.get(url)  # Асинхронный запрос
            response.raise_for_status()  # Проверка на ошибки

            # Парсинг HTML с помощью BeautifulSoup
            soup = BeautifulSoup(
                response.text, "html.parser"
            )  # Указание парсера, например, 'html.parser'
            
            day = soup.find_all("div", class_="grid")
            
            # Вычленение первичного контента
            day_content = []
            
            for i in range(len(day)):
                day_content.append(day[i].contents)
            
            print(day_content)

            # Первичное преобразование
            for container in day_content:
                for element in range(len(container)):
                    try:
                        container[element] = (
                            str(container[element])
                            .replace("<div>", "")
                            .replace("</div>", "")
                            .replace("<!-- -->", "")
                        )
                        container[element]
                    except Exception as e:
                        print(e)
            
            print(day_content)

            # Удалим неинформативные строки
            day_content = day_content[1:]
            day_content.pop(1)

            # Преобразование дат и вычлинение значений
            for element in day_content:
                try:
                    # Преобразованиие даты
                    element[0] = pd.to_datetime(element[0]).strftime("%Y-%m-%d")

                    # Числовые преобразования
                    element[1] = pd.to_numeric(
                        element[1].split("/")[1].strip().replace(",", "")
                    )
                    element[2] = pd.to_numeric(
                        element[2].split("/")[0].strip().replace(",", "")
                    )
                    element[3] = pd.to_numeric(
                        element[3].split("/")[1].strip().replace(",", "")
                    )
                    element[4] = pd.to_numeric(
                        element[4].split("/")[1].strip().replace(",", "")
                    )
                except Exception as e:
                    print(e)
                    pass
            
            print(day_content)

            # Формируем датафрейм
            day_content = pd.DataFrame(
                day_content, columns=["Date", "Gold", "Silver", "Platinum", "Palladium"]
            )

            # Преобразуем типы
            day_content["Date"] = pd.to_datetime(day_content["Date"])
            day_content = day_content.sort_values("Date").reset_index(drop=True)

            # Возьмем последние 5 строк
            day_update = day_content.tail()
            
            # Склеим с историческими данными
            historical_df = pd.read_excel("./data/kitko_db.xlsx", index_col=0)
            historical_df = (
                pd.concat([historical_df, day_update], axis=0)
                .reset_index(drop=True)
                .drop_duplicates()
            )

            with pd.ExcelWriter(
                        "./data/kitko_db.xlsx",
                        date_format="YYYY-MM-DD",
                        datetime_format="YYYY-MM-DD",
                    ) as writer:
                        historical_df.to_excel(writer, sheet_name="kitco_metall")

            print("KITKO_main is done!!!")
            # display(historical_df.tail(10))

            return historical_df

    try:
        # Получаем данные асинхронно
        gold = await get_raw_data(url)
        # display(gold)

    except Exception as error:
        print("Произошла ошибка:", error)


# Запускаем асинхронную функцию
await kitco_parser_async()  # Просто вызываем, без asyncio.run()


[[<div class="flex items-center justify-center">DATE</div>, <h4>Gold <br/> <span class="text-sm text-gray-600 font-normal">AM/PM</span></h4>, <h4>Silver <br/> <span class="text-sm text-gray-600 font-normal">NOON</span></h4>, <h4>Platinum <br/> <span class="text-sm text-gray-600 font-normal">AM/PM</span></h4>, <h4>Palladium <br/> <span class="text-sm text-gray-600 font-normal">AM/PM</span></h4>], [<div class="flex justify-center"><div class="animate-loading !h-[16px] !w-[50%] !bg-black/10"></div></div>, <div class="flex justify-center"><div class="animate-loading !h-[16px] !w-[50%] !bg-black/10"></div></div>, <div class="flex justify-center"><div class="animate-loading !h-[16px] !w-[50%] !bg-black/10"></div></div>, <div class="flex justify-center"><div class="animate-loading !h-[16px] !w-[50%] !bg-black/10"></div></div>, <div class="flex justify-center"><div class="animate-loading !h-[16px] !w-[50%] !bg-black/10"></div></div>], [<div class="flex items-center justify-center">DATE</div>, 

---
### Antimony

In [8]:
url = "https://www.metal.com/Antimony/201102250328"

service = Service()
driver = webdriver.Chrome(service=service)

try:
    driver.maximize_window()
    driver.get(url=url)
    time.sleep(2)

    driver.execute_script("window.scrollTo(0, window.scrollY + window.innerHeight);")
    driver.execute_script("window.scrollTo(0, window.scrollY + window.innerHeight);")

    time.sleep(2)

    html_code = driver.page_source

    soup = BeautifulSoup(html_code, "html.parser")

    responce = [
        float(i.text.replace(",", ""))
        for i in soup.find_all("div", class_="price___2mpJr")
    ][1:][::-1]

    responce_date = soup.find("div", class_="date___2QLQE")
    responce_date = pd.to_datetime(responce_date.text)

    for i in responce:
        responce_date.append(i)

    result = pd.DataFrame(responce_date).T.set_axis(
        ["Date", "Avg(CNY/mt,VAT included)", "Avg With Rate(USD/mt,VAT included)"],
        axis=1,
    )


except Exception as error:
    print(error)

In [79]:
responce = [
    float(i.text.replace(",", "")) for i in soup.find_all("div", class_="price___2mpJr")
][1:]

cny_price = responce[1]
usd_price = responce[0]

print(cny_price, usd_price)

145000.0 19837.74


In [80]:
responce_date = soup.find("div", class_="date___2QLQE")
responce_date = [pd.to_datetime(responce_date.text)]
responce_date

[Timestamp('2025-01-24 00:00:00')]

In [81]:
result = pd.DataFrame(
    {
        "Date": responce_date,
        "Avg(CNY/mt,VAT included)": cny_price,
        "Avg With Rate(USD/mt,VAT included)": usd_price,
    }
)

,Date,"Avg(CNY/mt,VAT included)","Avg With Rate(USD/mt,VAT included)"
0,2025-01-24,145000.0,19837.74


In [67]:
result = pd.DataFrame(responce_date).T.set_axis(
    ["Date", "Avg(CNY/mt,VAT included)", "Avg With Rate(USD/mt,VAT included)"], axis=1
)

,Date,"Avg(CNY/mt,VAT included)","Avg With Rate(USD/mt,VAT included)"
0,2025-01-24 00:00:00,145000.0,19837.74


,Date,"Avg(CNY/mt,VAT included)","Avg With Rate(USD/mt,VAT included)"
0,2024-01-02,84000,11761.41
1,2024-01-03,84000,11728.57
2,2024-01-04,84000,11712.21
3,2024-01-05,85000,11832.51
4,2024-01-08,85500,11923.02
...,...,...,...
254,2025-01-20,145000,19751.27
255,2025-01-21,145000,19882.90
256,2025-01-22,145000,19882.90
257,2025-01-23,145000,19878.81


In [ ]:
historical_df.concat()

In [101]:
async def antimony():
    url = "https://www.metal.com/Antimony/201102250328"

    async def get_raw_data(url):
        async with httpx.AsyncClient() as client:
            response = await client.get(url)  # Асинхронный запрос
            response.raise_for_status()  # Проверка на ошибки

            # Парсинг HTML с помощью BeautifulSoup
            soup = BeautifulSoup(response.text, "html.parser")

            # Извлечение цены
            prices = [
                float(i.text.replace(",", ""))
                for i in soup.find_all("div", class_="price___2mpJr")
            ][1:]
            
            cny_price = prices[1]
            usd_price = prices[0]

            # Извлечение даты
            response_date = soup.find("div", class_="date___2QLQE")
            date = [pd.to_datetime(response_date.text)]

            # Создание DataFrame
            result = pd.DataFrame(
                {
                    "Date": date,
                    "Avg(CNY/mt,VAT included)": cny_price,
                    "Avg With Rate(USD/mt,VAT included)": usd_price,
                }
            )
            
            historical_df = pd.read_excel("./data/antimony.xlsx")
            
            historical_df = (
                pd.concat([historical_df, result], axis=0)
                .reset_index(drop=True)
                .drop_duplicates()
            )
            
            with pd.ExcelWriter(
                        "./data/antimony.xlsx",
                        date_format="YYYY-MM-DD",
                        datetime_format="YYYY-MM-DD",
                    ) as writer:
                        historical_df.to_excel(writer, sheet_name="antimony", index=False)

            print("antimony is done!!!")

            return historical_df

    try:
        # Получаем данные асинхронно
        antimony_data = await get_raw_data(url)
        print(antimony_data)  # Используйте print для отображения результата

    except Exception as error:
        print("Произошла ошибка:", str(error))

# Запускаем асинхронную функцию
if __name__ == "__main__":
    await antimony() # Используйте asyncio.run для запуска в обычном скрипте

antimony is done!!!
          Date  Avg(CNY/mt,VAT included)  Avg With Rate(USD/mt,VAT included)
0   2024-01-02                   84000.0                            11761.41
1   2024-01-03                   84000.0                            11728.57
2   2024-01-04                   84000.0                            11712.21
3   2024-01-05                   85000.0                            11832.51
4   2024-01-08                   85500.0                            11923.02
..         ...                       ...                                 ...
254 2025-01-20                  145000.0                            19751.27
255 2025-01-21                  145000.0                            19882.90
256 2025-01-22                  145000.0                            19882.90
257 2025-01-23                  145000.0                            19878.81
258 2025-01-24                  145000.0                            19837.74

[259 rows x 3 columns]


----


In [20]:
# Main import block
import asyncio
import datetime
import signal
import time
import warnings
from contextlib import contextmanager
from datetime import date, datetime
from io import StringIO
import os

# Sneaky scrapper
import cfscrape
# Upload to google sheets
import gspread
# import df2gspread as d2g
import gspread_pandas as gsp
import numpy as np
import pandas as pd
import httpx
import requests
# Upload to yandex
import yadisk
from bs4 import BeautifulSoup
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

warnings.filterwarnings("ignore")
################################################################
##########################   LME ###############################
################################################################
# Асинхронная функция
def lme_selenium_async():
    url = "https://www.lme.com/Metals/Non-ferrous#tabIndex=1"
    #url = "https://www.lme.com/api/trading-data/metal-block?datasourceIds=611e28f8-d8c3-4529-9faa-fac7645f5cc0&datasourceIds=4a1f4a55-000a-4667-beab-d71041ffee19&datasourceIds=e870f5f2-aec3-4854-b349-00cba2785b5a&datasourceIds=9962a3b5-a22f-4a79-85fb-7b21c363da67&datasourceIds=26b55bff-1587-412d-8403-1a66e68b8824&datasourceIds=af7afbb2-373f-4c0e-bb29-47487ba2361b&datasourceIds=47a1e446-a34e-4978-916e-abe60a9e5dd7&datasourceIds=48fe37ba-aa51-47c1-9563-21cfd03cc355&datasourceIds=4b620e86-85bd-4500-b0b8-20d1c7ed1ba4&datasourceIds=40a17f28-449f-428a-ae2a-bd65ce582618&datasourceIds=2def21a6-6b67-4c0f-974e-afb83c5e3bfb&datasourceIds=36505bb5-09f3-47e3-aaa8-17e68ecb147a&datasourceIds=7bd3beb0-d5e3-4713-aebb-ab93c2a75037"

    service = Service()
    options = webdriver.ChromeOptions()
    options.add_argument('headless') # без графического интерфейса
    options.add_argument('--no-sandbox')  # для будущей контейнеризации
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    #driver = webdriver.Chrome(service=service, options=options)
    driver = webdriver.Chrome(service=service)

    try:
        # driver.maximize_window()
        driver.get(url=url)

        time.sleep(10)

        # driver.execute_script(
        #     "window.scrollTo(0, window.scrollY + window.innerHeight);"
        # )
        # driver.execute_script(
        #     "window.scrollTo(0, window.scrollY + window.innerHeight);"
        # )

        # time.sleep(2)

        html_code = driver.page_source
        # driver.close()

        soup = BeautifulSoup(html_code, "html.parser")
        # Вытаскиваем данные из табличного блока
        data_raw = soup.find_all("div", class_="metal-block-row__blocks")

        # Приводим к тексту
        metalls_raw = data_raw[0].text

        # # Убираем все лишнее
        # metalls_raw = metalls_raw.replace(
        #     " ", "").replace("LME", "LME_").split("\n")
        # metalls_raw = " ".join(metalls_raw).strip().split(" ")

        # # Конфигурируем новый лист по металлам
        # metalls_raw = [metalls_raw[i: i + 4]
        #                for i in range(0, len(metalls_raw), 4)]

        # # Обрезаем до нужной инфы
        # metalls_raw = [i[:2] for i in metalls_raw]

        # # Приведем цены к числовому формату
        # for metall in metalls_raw:
        #     metall[1] = pd.to_numeric(metall[1])

        # # Формируем строку датафрейма по металлам
        # dict_list = [{item[0]: item[1]} for item in metalls_raw]

        # metall_df = pd.DataFrame()

        # for i in dict_list:
        #     keys = list(i.keys())
        #     values = list(i.values())
        #     metall_df[keys[0]] = values

        # # Дропаем ненужные колонки
        # metall_df.drop(columns=["LME_AluminiumAlloy",
        #                "LME_NASAAC"], inplace=True)
        # metall_df = metall_df.rename(
        #     columns={
        #         "LME_Aluminium": "aluminium",
        #         "LME_Copper": "copper",
        #         "LME_Lead": "lead",
        #         "LME_Nickel": "nickel",
        #         "LME_Zinc": "zink",
        #         "LME_Tin": "tin",
        #     }
        # )

        # # Достанем в вставим дату
        # date_raw = soup.find_all(
        #     "span", class_="metal-block-container__refreshed-on")
        # date_raw = date_raw[0]
        # date_raw = str(date_raw).split(">")[1]
        # date_raw = date_raw.replace("\xa0", "").replace("</span", "")
        # date_raw = pd.to_datetime(date_raw) - pd.Timedelta(days=1)

        # metall_df["date"] = date_raw

        # # Поменяем порядок колонок
        # new_column_order = [
        #     "date",
        #     "aluminium",
        #     "copper",
        #     "lead",
        #     "nickel",
        #     "zink",
        #     "tin",
        # ]
        # metall_df = metall_df[new_column_order]
        return metalls_raw,# dict_list, metall_df
        
    except Exception as error:
        print(error)

    finally:
        driver.close()
        driver.quit()

In [21]:
metalls_raw = lme_selenium_async()

In [23]:
print(metalls_raw)

('\n\n\n\n\n\n                    LME Aluminium\n                \n\n\n{{ price }}\n\n\n\n                    {{ change }}\n                \n\nOfficial Price\n{{ expiryMessage }} {{ expiryDate }}\n\n\n\n\n\n\n\n\n                    LME Copper\n                \n\n\n{{ price }}\n\n\n\n                    {{ change }}\n                \n\nOfficial Price\n{{ expiryMessage }} {{ expiryDate }}\n\n\n\n\n\n\n\n\n                    LME Zinc\n                \n\n\n{{ price }}\n\n\n\n                    {{ change }}\n                \n\nOfficial Price\n{{ expiryMessage }} {{ expiryDate }}\n\n\n\n\n\n\n\n\n                    LME Nickel\n                \n\n\n{{ price }}\n\n\n\n                    {{ change }}\n                \n\nOfficial Price\n{{ expiryMessage }} {{ expiryDate }}\n\n\n\n\n\n\n\n\n                    LME Lead\n                \n\n\n{{ price }}\n\n\n\n                    {{ change }}\n                \n\nOfficial Price\n{{ expiryMessage }} {{ expiryDate }}\n\n\n\n\n\n\n\n\n  

---


In [2]:
import requests
import pandas as pd

In [3]:
url = "https://www.lme.com/api/trading-data/metal-block?datasourceIds=611e28f8-d8c3-4529-9faa-fac7645f5cc0&datasourceIds=4a1f4a55-000a-4667-beab-d71041ffee19&datasourceIds=e870f5f2-aec3-4854-b349-00cba2785b5a&datasourceIds=9962a3b5-a22f-4a79-85fb-7b21c363da67&datasourceIds=26b55bff-1587-412d-8403-1a66e68b8824&datasourceIds=af7afbb2-373f-4c0e-bb29-47487ba2361b&datasourceIds=47a1e446-a34e-4978-916e-abe60a9e5dd7&datasourceIds=48fe37ba-aa51-47c1-9563-21cfd03cc355&datasourceIds=4b620e86-85bd-4500-b0b8-20d1c7ed1ba4&datasourceIds=40a17f28-449f-428a-ae2a-bd65ce582618&datasourceIds=2def21a6-6b67-4c0f-974e-afb83c5e3bfb&datasourceIds=36505bb5-09f3-47e3-aaa8-17e68ecb147a&datasourceIds=7bd3beb0-d5e3-4713-aebb-ab93c2a75037"

In [4]:
responce = requests.get(url=url)

In [6]:
responce.text

'<!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}.main-content{margin:8rem auto;max-width:60rem;padding-left:1.5rem}@media (width <= 720px){.main-content{margin-top:4rem}}.h2{font-size:1.5rem;font-weight:500;line-height:2.25rem}@media (width <= 720px){.h2{font-size:1.25rem;line-height:1.5rem}}#challenge-error-text{background-image:url();background-repeat:no-repeat;background-

In [ ]:
import tensorflow as tf

# Проверка доступности GPU
gpu_devices = tf.config.list_physical_devices('GPU')
print("GPU Devices:", gpu_devices)

# Проверка, что TensorFlow использует GPU
if gpu_devices:
    print("Metal активирован! 🚀")
else:
    print("GPU не обнаружен. Проверьте установку.")

GPU Devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Metal активирован! 🚀


In [ ]:
# Создаем тензор и выполняем операцию
with tf.device('/GPU:0'):
    a = tf.constant([1.0, 2.0])
    b = tf.constant([3.0, 4.0])
    c = tf.add(a, b)
    print("Результат (GPU):", c.numpy())

# Если ошибок нет — Metal работает.

2025-04-26 19:08:45.140872: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-04-26 19:08:45.141136: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-04-26 19:08:45.141156: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-04-26 19:08:45.141351: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-26 19:08:45.141745: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Результат (GPU): [4. 6.]


In [ ]:
import time
import tensorflow as tf

# Создаем большой тензор
data = tf.random.normal(shape=(10000, 10000))

# На CPU
start = time.time()
with tf.device('/CPU:0'):
    tf.reduce_sum(data)
print(f"CPU: {time.time() - start:.2f} сек")

# На GPU (Metal)
start = time.time()
with tf.device('/GPU:0'):
    tf.reduce_sum(data)
print(f"GPU: {time.time() - start:.2f} сек")

2025-04-26 19:12:07.696128: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-04-26 19:12:07.696307: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-04-26 19:12:07.696313: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-04-26 19:12:07.696797: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-26 19:12:07.696808: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


CPU: 0.18 сек
GPU: 0.13 сек


In [2]:
# Free proxy function
def get_free_proxies():
    url = "https://free-proxy-list.net/"
    soup = BeautifulSoup(requests.get(url).content, "html.parser")

    raw_list = []
    proxies = dict()
    trs = soup.find("table").find_all("tr")  # main table

    for i in trs[1:]:
        raw_list.append(i.find_all("td"))  # list of raw data rows

    for i in range(len(raw_list)):  # creating working proxy list
        try:
            if raw_list[i][6].text == "yes":  # taking only https
                proxies[raw_list[i][3].text] = (
                        f"{raw_list[i][0].text}:{raw_list[i][1].text}"
                    )
        except IndexError:
            continue

    adress = pd.Series(proxies)  # creating proxy series

    return adress

In [ ]:

################################################################
##########################   LME ###############################
################################################################
# Асинхронная функция
def lme_selenium_async(random_proxy=True):
    url = "https://www.lme.com/Metals/Non-ferrous#tabIndex=1"
    
    # service = Service()
    # options = webdriver.ChromeOptions()
    # options.add_argument('headless') # без графического интерфейса
    # options.add_argument('--no-sandbox')  # для будущей контейнеризации
    # options.add_argument(
    #     "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    # driver = webdriver.Chrome(service=service, options=options)
    
    if random_proxy:
        # Получаем список прокси
        proxies = get_free_proxies()
        print(f"Найдено {len(proxies)} HTTPS-прокси")
        
        driver = None
        timeout_seconds = 7
        max_attempts = len(proxies) + 1  # +1 - чтобы попробовать без прокси в конце
        attempt = 0

        while attempt < max_attempts:
            options = webdriver.ChromeOptions()
            options.add_argument('headless')
            options.add_argument('--no-sandbox')
            options.add_argument(
                "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
            )

            proxy = None
            if attempt < len(proxies):
                proxy = proxies[attempt]
                print(f"Попытка с прокси: {proxy}")
                options.add_argument(f'--proxy-server={proxy}')
            else:
                print("Попытка без использования прокси")

            try:
                service = Service()
                driver = webdriver.Chrome(service=service, options=options)

                # Устанавливаем общий тайм-аут ожидания
                driver.set_page_load_timeout(timeout_seconds)
                
                # Попытка загрузки страницы
                driver.get(url)

                # Если всё прошло успешно - выходим из цикла
                print("Успешное подключение!")
                break

            except (TimeoutException, WebDriverException) as e:
                print(f"Ошибка при подключении (попытка {attempt + 1}): {e}")
                if driver:
                    driver.quit()
                attempt += 1
                continue

        if not driver:
            print("Не удалось подключиться ни через один прокси.")
            return

    else:
        service = Service()
        options = webdriver.ChromeOptions()
        options.add_argument('headless') # без графического интерфейса
        options.add_argument('--no-sandbox')  # для будущей контейнеризации
        options.add_argument(
            "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
        driver = webdriver.Chrome(service=service, options=options)
    
    
    # driver = webdriver.Chrome(service=service)

    try:
        # driver.maximize_window()
        driver.get(url=url)

        time.sleep(10)

        driver.execute_script(
            "window.scrollTo(0, window.scrollY + window.innerHeight);"
        )
        driver.execute_script(
            "window.scrollTo(0, window.scrollY + window.innerHeight);"
        )

        time.sleep(2)

        html_code = driver.page_source
        # driver.close()

        soup = BeautifulSoup(html_code, "html.parser")
        # Вытаскиваем данные из табличного блока
        data_raw = soup.find_all("div", class_="metal-block-row__blocks")
        
        return data_raw, html_code

        print("LME_main is done!!!")
        # display(lme_db.tail(10))

    except Exception as error:
        print(error)

    finally:
        driver.close()
        driver.quit()

In [9]:
data_raw, html_code = lme_selenium_async(random_proxy=False)

In [10]:
# Приводим к тексту
metalls_raw = data_raw[0].text
metalls_raw

'\n\n\n\n\n\n                    LME Aluminium\n                \n\n\n{{ price }}\n\n\n\n                    {{ change }}\n                \n\nOfficial Price\n{{ expiryMessage }} {{ expiryDate }}\n\n\n\n\n\n\n\n\n                    LME Copper\n                \n\n\n{{ price }}\n\n\n\n                    {{ change }}\n                \n\nOfficial Price\n{{ expiryMessage }} {{ expiryDate }}\n\n\n\n\n\n\n\n\n                    LME Zinc\n                \n\n\n{{ price }}\n\n\n\n                    {{ change }}\n                \n\nOfficial Price\n{{ expiryMessage }} {{ expiryDate }}\n\n\n\n\n\n\n\n\n                    LME Nickel\n                \n\n\n{{ price }}\n\n\n\n                    {{ change }}\n                \n\nOfficial Price\n{{ expiryMessage }} {{ expiryDate }}\n\n\n\n\n\n\n\n\n                    LME Lead\n                \n\n\n{{ price }}\n\n\n\n                    {{ change }}\n                \n\nOfficial Price\n{{ expiryMessage }} {{ expiryDate }}\n\n\n\n\n\n\n\n\n   

In [ ]:


        # Убираем все лишнее
metalls_raw = metalls_raw.replace(
    " ", "").replace("LME", "LME_").split("\n")
metalls_raw = " ".join(metalls_raw).strip().split(" ")

In [22]:
metalls_raw

['LME_Aluminium',
 '2510.00',
 '-0.06%',
 'OfficialPrice',
 'LME_Copper',
 '9779.00',
 '+0.5%',
 'OfficialPrice',
 'LME_Zinc',
 '2615.00',
 '-0.68%',
 'OfficialPrice',
 'LME_Nickel',
 '14940.00',
 '-0.6%',
 'OfficialPrice',
 'LME_Lead',
 '1966.00',
 '+0.43%',
 'OfficialPrice',
 'LME_Tin',
 '32725.00',
 '+0.09%',
 'OfficialPrice',
 'LME_AluminiumAlloy',
 '2509.00',
 '+0%',
 'OfficialPrice',
 'LME_NASAAC',
 '2400.00',
 '+0%',
 'OfficialPrice']

---
---
## Antimony new parser

In [25]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

In [66]:
# Текущая дата
end_date = time.strftime("%Y-%m-%d")

# Преобразуем строку в datetime
date_obj = datetime.strptime(end_date, "%Y-%m-%d")

# Вычитаем 7 дней
new_date = date_obj - timedelta(days=12)
begin_date = new_date.strftime("%Y-%m-%d")

begin_date, end_date
begin_date = '2026-02-01'
begin_date

'2026-02-01'

In [67]:
url = f'https://platform.metal.com/spotoverseascenter/v1/product_info/history/201102250328?product_id=201102250328&begin_date={begin_date}&end_date={end_date}&line_type=d&auth_token=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJjZWxscGhvbmUiOiIiLCJjb21wYW55X2lkIjowLCJjb21wYW55X3N0YXR1cyI6MCwiY3JlYXRlX2F0IjoxNzcwNjAyNjk3LCJlbWFpbCI6InNub3d5ZGF5NkBnbWFpbC5jb20iLCJlbl9lbmRfdGltZSI6MCwiZW5fcmVnaXN0ZXJfc3RlcCI6MSwiZW5fcmVnaXN0ZXJfdGltZSI6MCwiZW5fc3RhcnRfdGltZSI6MCwiZW5fdXNlcl90eXBlIjowLCJlbmRfdGltZSI6MCwiaXNfbWFpbCI6MCwiaXNfcGhvbmUiOjAsImxhbmd1YWdlIjoiIiwibHlfZW5kX3RpbWUiOjAsImx5X3N0YXJ0X3RpbWUiOjAsImx5X3VzZXJfdHlwZSI6MCwib25saW5lX2FwcCI6IiIsInJlZ2lzdGVyX3RpbWUiOjE3Mzc3MDYzOTgsInN0YXJ0X3RpbWUiOjAsInVuaXF1ZV9pZCI6IjU3MmI3ODU1MWJhYzNiOTE1NDZhYWY3YTAzOTU3ZTc5IiwidXNlcl9pZCI6Mzk4NDQ1OCwidXNlcl9sYW5ndWFnZSI6ImVuIiwidXNlcl9uYW1lIjoiU01NMTczNzcwNjM5OVdmIiwidXNlcl90eXBlIjowLCJ1dWlkX3NoYTI1NiI6ImMyMjM1ZGQ3MTE1ZjI5NzRiNWM0OTllYWI0ZTBlNWMyZTMyNDYwNWFmYzRiOTE2ZGY0MDg1YjZmOGIwZGJhMTEiLCJ6eF9lbmRfdGltZSI6MCwienhfc3RhcnRfdGltZSI6MCwienhfdXNlcl90eXBlIjowfQ.UpdE8W4cvohelPaHNO5t3m7CoKu3svNUjiCwijz-TGA&language=ru'

In [68]:
responce = requests.get(url, json=True)
responce_body = responce.json()

In [75]:
columns = ['renew_date', "average",]
responce_df = pd.DataFrame(responce_body['data']['prices'])[columns]
responce_df.rename(columns={'renew_date':'Date','average':'Avg(CNY/mt,VAT included)'}, inplace=True)
responce_df["Avg With Rate(USD/mt,VAT included)"] = None
responce_df

,Date,"Avg(CNY/mt,VAT included)","Avg With Rate(USD/mt,VAT included)"
0,2026-02-09,168000,None
1,2026-02-06,148008,None
2,2026-02-05,147127,None
3,2026-02-04,146246,None
4,2026-02-03,146246,None
5,2026-02-02,145365,None


In [70]:
data_df = pd.read_csv('./data/antimony.csv')
data_df

,Date,"Avg(CNY/mt,VAT included)","Avg With Rate(USD/mt,VAT included)"
0,2024-01-02,84000,11761.41
1,2024-01-03,84000,11728.57
2,2024-01-04,84000,11712.21
3,2024-01-05,85000,11832.51
4,2024-01-08,85500,11923.02
...,...,...,...
470,2026-01-26,165000,23661.68
471,2026-01-27,165000,23659.31
472,2026-01-28,165000,23684.10
473,2026-01-29,165000,23684.10


In [76]:
new_df = pd.concat([data_df, responce_df], axis=0).reset_index(drop=True)
new_df.sort_values(by="Date")

/var/folders/tj/8gvnjzsd1zg6q14665cymt1r0000gp/T/ipykernel_39016/2874052353.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([data_df, responce_df], axis=0).reset_index(drop=True)


,Date,"Avg(CNY/mt,VAT included)","Avg With Rate(USD/mt,VAT included)"
0,2024-01-02,84000,11761.41
1,2024-01-03,84000,11728.57
2,2024-01-04,84000,11712.21
3,2024-01-05,85000,11832.51
4,2024-01-08,85500,11923.02
...,...,...,...
479,2026-02-03,146246,NaN
478,2026-02-04,146246,NaN
477,2026-02-05,147127,NaN
476,2026-02-06,148008,NaN


In [1]:
###############################################################
########################   antimony (NEW) ###########################
###############################################################

async def antimony():
    # Текущая дата
    end_date = time.strftime("%Y-%m-%d")

    # Преобразуем строку в datetime
    date_obj = datetime.strptime(end_date, "%Y-%m-%d")

    # Вычитаем 7 дней
    new_date = date_obj - timedelta(days=12)
    begin_date = new_date.strftime("%Y-%m-%d")

    # begin_date, end_date
    # begin_date = '2026-02-01' 
    
    url = url = f'https://platform.metal.com/spotoverseascenter/v1/product_info/history/201102250328?product_id=201102250328&begin_date={begin_date}&end_date={end_date}&line_type=d&auth_token=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJjZWxscGhvbmUiOiIiLCJjb21wYW55X2lkIjowLCJjb21wYW55X3N0YXR1cyI6MCwiY3JlYXRlX2F0IjoxNzcwNjAyNjk3LCJlbWFpbCI6InNub3d5ZGF5NkBnbWFpbC5jb20iLCJlbl9lbmRfdGltZSI6MCwiZW5fcmVnaXN0ZXJfc3RlcCI6MSwiZW5fcmVnaXN0ZXJfdGltZSI6MCwiZW5fc3RhcnRfdGltZSI6MCwiZW5fdXNlcl90eXBlIjowLCJlbmRfdGltZSI6MCwiaXNfbWFpbCI6MCwiaXNfcGhvbmUiOjAsImxhbmd1YWdlIjoiIiwibHlfZW5kX3RpbWUiOjAsImx5X3N0YXJ0X3RpbWUiOjAsImx5X3VzZXJfdHlwZSI6MCwib25saW5lX2FwcCI6IiIsInJlZ2lzdGVyX3RpbWUiOjE3Mzc3MDYzOTgsInN0YXJ0X3RpbWUiOjAsInVuaXF1ZV9pZCI6IjU3MmI3ODU1MWJhYzNiOTE1NDZhYWY3YTAzOTU3ZTc5IiwidXNlcl9pZCI6Mzk4NDQ1OCwidXNlcl9sYW5ndWFnZSI6ImVuIiwidXNlcl9uYW1lIjoiU01NMTczNzcwNjM5OVdmIiwidXNlcl90eXBlIjowLCJ1dWlkX3NoYTI1NiI6ImMyMjM1ZGQ3MTE1ZjI5NzRiNWM0OTllYWI0ZTBlNWMyZTMyNDYwNWFmYzRiOTE2ZGY0MDg1YjZmOGIwZGJhMTEiLCJ6eF9lbmRfdGltZSI6MCwienhfc3RhcnRfdGltZSI6MCwienhfdXNlcl90eXBlIjowfQ.UpdE8W4cvohelPaHNO5t3m7CoKu3svNUjiCwijz-TGA&language=ru'

    async def get_raw_data(url):
        async with httpx.AsyncClient() as client:
            
            responce = await client.get(url)
            responce_body = responce.json()
            
            columns = ['renew_date', "average",]
            responce_df = pd.DataFrame(responce_body['data']['prices'])[columns]
            responce_df.rename(columns={'renew_date':'Date','average':'Avg(CNY/mt,VAT included)'}, inplace=True)
            responce_df["Avg With Rate(USD/mt,VAT included)"] = None
            
            data_df = pd.read_csv('./data/antimony.csv')
            
            new_df = pd.concat([data_df, responce_df], axis=0).reset_index(drop=True)
            new_df = new_df.sort_values(by="Date").drop_duplicates()
            new_df.to_csv("./data/antimony.csv", index=False)
            
            with pd.ExcelWriter(
                        "./data/antimony.xlsx",
                        date_format="YYYY-MM-DD",
                        datetime_format="YYYY-MM-DD",
                    ) as writer:
                        new_df.to_excel(writer, sheet_name="antimony")
            
            
            return historical_df

    try:
        # Получаем данные асинхронно
        antimony = await get_raw_data(url)
        print("antimony is done!!!")

    except Exception as error:
        print("Произошла ошибка:", str(error))

# # Запускаем асинхронную функцию
# if __name__ == "__main__":
#     await antimony() # Используйте asyncio.run для запуска в обычном скрипте

In [ ]:
usd_url = 'https://platform.metal.com/spotoverseascenter/v1/product_info/history/201102250328?product_id=201102250328&begin_date=2026-02-02&end_date=2026-02-27&line_type=d&auth_token=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJjZWxscGhvbmUiOiIiLCJjb21wYW55X2lkIjowLCJjb21wYW55X3N0YXR1cyI6MCwiY3JlYXRlX2F0IjoxNzcwNjAyNjk3LCJlbWFpbCI6InNub3d5ZGF5NkBnbWFpbC5jb20iLCJlbl9lbmRfdGltZSI6MCwiZW5fcmVnaXN0ZXJfc3RlcCI6MSwiZW5fcmVnaXN0ZXJfdGltZSI6MCwiZW5fc3RhcnRfdGltZSI6MCwiZW5fdXNlcl90eXBlIjowLCJlbmRfdGltZSI6MCwiaXNfbWFpbCI6MCwiaXNfcGhvbmUiOjAsImxhbmd1YWdlIjoiIiwibHlfZW5kX3RpbWUiOjAsImx5X3N0YXJ0X3RpbWUiOjAsImx5X3VzZXJfdHlwZSI6MCwib25saW5lX2FwcCI6IiIsInJlZ2lzdGVyX3RpbWUiOjE3Mzc3MDYzOTgsInN0YXJ0X3RpbWUiOjAsInVuaXF1ZV9pZCI6IjU3MmI3ODU1MWJhYzNiOTE1NDZhYWY3YTAzOTU3ZTc5IiwidXNlcl9pZCI6Mzk4NDQ1OCwidXNlcl9sYW5ndWFnZSI6ImVuIiwidXNlcl9uYW1lIjoiU01NMTczNzcwNjM5OVdmIiwidXNlcl90eXBlIjowLCJ1dWlkX3NoYTI1NiI6ImMyMjM1ZGQ3MTE1ZjI5NzRiNWM0OTllYWI0ZTBlNWMyZTMyNDYwNWFmYzRiOTE2ZGY0MDg1YjZmOGIwZGJhMTEiLCJ6eF9lbmRfdGltZSI6MCwienhfc3RhcnRfdGltZSI6MCwienhfdXNlcl90eXBlIjowfQ.UpdE8W4cvohelPaHNO5t3m7CoKu3svNUjiCwijz-TGA&language=ru'

In [ ]:
yuan_url = 'https://platform.metal.com/spotoverseascenter/v1/product_info/history/201102250328?product_id=201102250328&line_type=d&shfe_line_type=time_sharing&begin_date=2026-02-02&end_date=2026-02-27&currency_type=1&auth_token=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJjZWxscGhvbmUiOiIiLCJjb21wYW55X2lkIjowLCJjb21wYW55X3N0YXR1cyI6MCwiY3JlYXRlX2F0IjoxNzcyMTYzOTg2LCJlbWFpbCI6InNub3d5ZGF5NkBnbWFpbC5jb20iLCJlbl9lbmRfdGltZSI6MCwiZW5fcmVnaXN0ZXJfc3RlcCI6MSwiZW5fcmVnaXN0ZXJfdGltZSI6MCwiZW5fc3RhcnRfdGltZSI6MCwiZW5fdXNlcl90eXBlIjowLCJlbmRfdGltZSI6MCwiaXNfbWFpbCI6MCwiaXNfcGhvbmUiOjAsImxhbmd1YWdlIjoiIiwibHlfZW5kX3RpbWUiOjAsImx5X3N0YXJ0X3RpbWUiOjAsImx5X3VzZXJfdHlwZSI6MCwib25saW5lX2FwcCI6IiIsInJlZ2lzdGVyX3RpbWUiOjE3Mzc3MDYzOTgsInN0YXJ0X3RpbWUiOjAsInVuaXF1ZV9pZCI6IjU3MmI3ODU1MWJhYzNiOTE1NDZhYWY3YTAzOTU3ZTc5IiwidXNlcl9pZCI6Mzk4NDQ1OCwidXNlcl9sYW5ndWFnZSI6ImVuIiwidXNlcl9uYW1lIjoiU01NMTczNzcwNjM5OVdmIiwidXNlcl90eXBlIjowLCJ1dWlkX3NoYTI1NiI6ImMyMjM1ZGQ3MTE1ZjI5NzRiNWM0OTllYWI0ZTBlNWMyZTMyNDYwNWFmYzRiOTE2ZGY0MDg1YjZmOGIwZGJhMTEiLCJ6eF9lbmRfdGltZSI6MCwienhfc3RhcnRfdGltZSI6MCwienhfdXNlcl90eXBlIjowfQ.OMP2Ny6yC-FEoLjy17egF8rHXQeZtBasrHEsFyL0Z5s&language=ru'